In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

In [2]:
dataset=pd.read_csv("dataset.csv",header = 0)
dataset.head()

,name,npm,class
0,Diaz Ilyasa Azrurrafi Saiful,1806200154,Despro-1
1,Dio Fajrie Fadlullah,1806200324,Despro-1
2,Muhammad Miftah Faridh,1806148782,Despro-2
3,Abdul Fattah Ismail,1806200255,Despro-2
4,Helmi Arrazy,1806200141,Despro-1


In [3]:
from fuzzywuzzy import fuzz

def fuzzScore(dataset, namecol, datainput):
    dataset['scoreRatio'] = 0
    dataset['scoreSet'] = 0
    dataset['scoreSort'] = 0
    for index, rows in dataset.iterrows():
        data = rows[namecol]
        dataset.loc[index, 'scoreSort']   = fuzz.token_sort_ratio(data, datainput)
        dataset.loc[index, 'scoreSet']    = fuzz.token_set_ratio(data, datainput)
        dataset.loc[index, 'scoreRatio']  = fuzz.ratio(data, datainput)
    return dataset

In [4]:
def getFullname(dataset, namecol, datainput):
    check = fuzzScore(dataset, namecol, datainput)
    tester = {'type': [
        'scoreRatio',
        'scoreSet',
        'scoreSort',
        ],
        'score':0,
        'values':0
        }
    df = pd.DataFrame(tester, columns = ['type', 'score','values'])
    for index, test in df.iterrows():
        check.sort_values(by = test.type, ascending=False, ignore_index=True, inplace=True)
        df.loc[index, 'score'] =check[test.type][0]
        df.loc[index, 'values'] = str(check.head()[namecol].tolist())
    df.sort_values(by = 'score', ascending=False, ignore_index=True, inplace=True)
    fullName = df['values'][0].replace('\'','')[1:-1].split(',')[0]
    return fullName, df

In [5]:
inputName = 'alfarisi'
fullname, val = getFullname(dataset, 'name', inputName)
print('Best Output : ' + fullname)
print(val)

Best Output : Mohammad Salman Alfarisi
         type  score                                             values
0    scoreSet    100  ['Mohammad Salman Alfarisi', 'Alfian Badrul Is...
1  scoreRatio     58  ['Natalia Kristian', 'Mohammad Salman Alfarisi...
2   scoreSort     52  ['Alfian Badrul Isnan', 'Abdul Fattah Ismail',...


In [11]:
def dataFillFullName(datasetbase, namecol, inputdataset, nicknamecol, fullnamecol, state):
    inputdata = inputdataset.copy()
    dataset = datasetbase.copy()
    inputdata[fullnamecol] = 0
    length = len(inputdata)
    for index, rows in inputdata.iterrows():
        out, var = getFullname(dataset, namecol, rows[nicknamecol])
        inputdata.loc[index, fullnamecol] = out
        if(state):print('Check : ' + str(index + 1) + '/' + str(length) + ' | ' + rows[nicknamecol] + ' ~ ' + out)
    return inputdata

In [21]:
def dataFillCheckError(datatest, namecol, testcol, dataoutput, nicknamecol, fullnamecol, check):
    length = len(dataoutput)
    passed = 0
    for index, output in dataoutput.iterrows():
        scan = datatest.loc[datatest[testcol] == output[nicknamecol]][namecol].tolist()[0]
        if(scan == output[fullnamecol]):
            passed+=1
            if(check): print('PASS ' + str(passed) + '/' + str(length))
        elif(check):
            print('----------------------------------')
            print('FAILED on : ' + str(index))
            print('NickName  : ' + output[nicknamecol])
            print('Output    : ' + output[fullnamecol])
            print('Original  : ' + scan)
            print('----------------------------------')
    return 'Error  : {:.2f}%'.format((length-passed)/length*100)

In [16]:
dataTest=pd.read_csv("datatest.csv",header = 0)
dataTest.head()

,name,test1,test2,test3,test4,test5
0,Diaz Ilyasa Azrurrafi Saiful,diaz,dias,diyaz,diyas,saeful
1,Dio Fajrie Fadlullah,dio,deo,diyo,fajri,fadlul
2,Muhammad Miftah Faridh,miptah,farid,farit,miptah farid,parid
3,Abdul Fattah Ismail,abdol,pattah,fatah,fatta,patta
4,Helmi Arrazy,helmy,halmi,helm,elmi,arasi


In [19]:
test1 = dataFillFullName(dataTest, 'name', dataTest, 'test1', 'NAMA LENGKAP', False)
test2 = dataFillFullName(dataTest, 'name', dataTest, 'test2', 'NAMA LENGKAP', False)
test3 = dataFillFullName(dataTest, 'name', dataTest, 'test3', 'NAMA LENGKAP', False)
test4 = dataFillFullName(dataTest, 'name', dataTest, 'test4', 'NAMA LENGKAP', False)

In [22]:
print(dataFillCheckError(dataTest, 'name', 'test1', test1, 'test1', 'NAMA LENGKAP', False))
print(dataFillCheckError(dataTest, 'name', 'test2', test2, 'test2', 'NAMA LENGKAP', False))
print(dataFillCheckError(dataTest, 'name', 'test3', test3, 'test3', 'NAMA LENGKAP', False))
print(dataFillCheckError(dataTest, 'name', 'test4', test4, 'test4', 'NAMA LENGKAP', False))

Error  : 47.76%
Error  : 56.72%
Error  : 61.19%
Error  : 47.76%


In [23]:
inputData=pd.read_csv("input.csv",header = 0)
inputData.head()

,NAMA ACAK,NAMA LENGKAP
0,diaz,NaN
1,dio,NaN
2,miptah,NaN
3,abdol,NaN
4,helmy,NaN


In [25]:
outputData = dataFillFullName(dataTest, 'name', inputData, 'NAMA ACAK', 'NAMA LENGKAP', False)
print(dataFillCheckError(dataTest, 'name', 'test1', outputData, 'NAMA ACAK', 'NAMA LENGKAP', False))
outputData.head()

Error  : 47.76%


,NAMA ACAK,NAMA LENGKAP
0,diaz,Diaz Ilyasa Azrurrafi Saiful
1,dio,Dio Fajrie Fadlullah
2,miptah,Dimas Radhitya
3,abdol,Bryan Oliver
4,helmy,Sherly


In [26]:
#outputData.to_csv(r'export\output.csv', index = False)